# Historical "high school activities" data on Earlham College applicants (2018-'21)

- This script inputs four Excel sheets containing data on all Earlham College applicants from 2018-2021.  
- Each file has somewhere between 900-1500 rows, with each row having 56 columns of data - 40 of which contain information that students wrote on their applications regarding the names and types of clubs and extracurricular activities they did in high school.
- The end result is a new Excel file called 'Total Clubs 2018-2021' which is an ordered list of the most common clubs and activities that Earlham applicants were a part of, with their counts for each year (2018, 19, 20, 21) and a "Total Counts" column.

In [1]:
import pandas as pd
import os
from collections import Counter
import xlsxwriter

# Output = file name and delete output IF it already exists

output = 'Total Clubs 2018-2021.xlsx'

if os.path.exists(output):
            os.remove(output)

# Current directory

directory = os.getcwd()
        
# Create list titled 'Files' which contains the path for each of my four Excel sheets        

files = [filename for filename in os.listdir(directory) if filename.startswith("App")]

# Create new Df, and then turn it into a list called 'list_of_clubs' containing all of the HS clubs that we want
# to check, and making everything in 'list_of_clubs' lowercase and remove whitespaces

df2 = pd.read_excel('club_list.xlsx')
list_of_clubs = df2['Clubs'].str.strip().str.lower().tolist()

for i in files:
    
    # Inputting my Excel sheet(s) into a DF and fill null values with white space
    
    app = pd.read_excel(i).fillna(' ')
    
    # Drop my "Interest Type" columns - not needed
    
    [app.drop(i, axis = 1, inplace=True) for i in app.columns if "Type" in i]
    
    # Combine all rows from column.index["Interest Name":] into a list called unique_cols. Getting rid of white spaces, making lowercase
    
    unique_cols = (app[app.columns[app.columns.get_loc('Interest Name'):]].apply(lambda x: ''.join(x.dropna()
                    .astype(str)), axis=1).str.strip().str.lower().tolist())
    
    # For each of the following: Counting the number of occurences for every club in list_of_clubs if it shows up in
    # my Excel file.  Appending to new list named after the year the Excel file references
    
    if '2018' in i:
        _2018 = [l for l in list_of_clubs for c in unique_cols if l in c]

    elif '2019' in i:
        _2019 = [l for l in list_of_clubs for c in unique_cols if l in c]
                    
    elif '2020' in i:
        _2020 = [l for l in list_of_clubs for c in unique_cols if l in c]
                    
    elif '2021' in i:
        _2021 = [l for l in list_of_clubs for c in unique_cols if l in c]
        
# Adding up occurences from each of the four lists into a new 'total' list so I can add a 'Total' column to Excel

total = _2018 + _2019 + _2020 + _2021

# Creating a new dictionary to store my values.  Keys = club names and values = # of times that club was listed on an application

clubs_and_counts = {}

# Adding each year and then the total count to my dictionary as values.  Keys are the club names

for l in list_of_clubs:
    clubs_and_counts[l] = [_2021.count(l), _2020.count(l), _2019.count(l), _2018.count(l), total.count(l)]

# Getting rid of dictinoary entries where 'Total' == 0

[clubs_and_counts.pop(k) for k in clubs_and_counts.copy() if clubs_and_counts.get(k)[-1] == 0]
        
# Creating a new DF from my dictionary

sorted_clubs = pd.DataFrame.from_dict(clubs_and_counts, orient='index')

# Labeling my columns

sorted_clubs.columns = ['2021', '2020','2019', '2018', 'Total']

# Creating my writer variable

writer = pd.ExcelWriter(output)

# Using pd.to_Excel to send my DF to Excel.  Sorting my values by 'Total' in reverse ascending order.  Naming
# my sheet 'EC Applicants Clubs_Activities' and my index 'Club Name'

sorted_clubs.sort_values(by='Total', ascending=False).to_excel(writer, sheet_name = 'EC Applicants Clubs_Activities', index_label = "Club Name")

# Manually setting column A to be a bit wider

worksheet = writer.sheets['EC Applicants Clubs_Activities']
worksheet.set_column('A:A', 25)

# Setting column F ('Total') to bold and freezing the top row and first column

workbook = xlsxwriter.Workbook(output)
cell_format = workbook.add_format()
cell_format.set_bold()
worksheet.set_column('F:F', None, cell_format)
worksheet.freeze_panes(1,1)

#Saving!

writer.save()
print('All done :-)')

All done :-)


In [4]:
sorted_clubs.sort_values('Total', ascending = False).head(25)

,2021,2020,2019,2018,Total
student government,161,230,165,225,781
dance,142,215,136,154,647
soccer,119,195,139,162,615
basketball,123,178,101,107,509
football,69,129,57,75,330
united nations,16,74,66,78,234
tennis,28,85,46,71,230
volleyball,48,82,43,48,221
student council,55,68,43,50,216
social media,40,38,33,37,148


In [ ]:
"""
With this program we now have a clear and reliable metric to show us exactly which clubs and programs
our applicants are most interested in.

Maybe we decide to start a program connecting our student government to student governments at key high schools,
or begin the process of bring a Model UN to our campus.

In the years that follow, we can continue to run this program to see if there has been an uptick in the
applicants from any of the clubs in which we are sponsoring programs.  This will also allow us to have a clear
and definitive answer of what's working and what isn't.
"""